# Univariate fine-mapping workflows

## Generate regional level genotype and phenotype data

### Genotype data partition by region

This step is necessary for finemapping using SuSiE.
`TADB_enhanced_cis.bed` is a list of TADB extended region for each gene, in formatting of gene_id, chr, start, end. The complete version of the file has been uploaded t
o [GitHub](https://github.com/cumc/fungen-xqtl-analysis/blob/main/resource/TADB_enhanced_cis.bed). Here we use a trimmed version that only contains information for the data we are analyzing in the MWE, ie, the `protocol_example.protein.enhanced_cis_chr21_chr22.bed` 

In [6]:
sos run pipeline/genotype_formatting.ipynb genotype_by_region \
    --region-list protocol_example/protocol_example.protein.enhanced_cis_chr21_chr22.bed \
    --genoFile protocol_example/protocol_example.genotype.chr21_22.bed \
    --cwd output \
    --container containers/bioinfo.sif

INFO: Running genotype_by_region_1: 
INFO: genotype_by_region_1 (index=1) is completed.
INFO: genotype_by_region_1 (index=4) is completed.
INFO: genotype_by_region_1 (index=3) is completed.
INFO: genotype_by_region_1 (index=5) is completed.
INFO: genotype_by_region_1 (index=0) is completed.
INFO: genotype_by_region_1 (index=2) is completed.
INFO: genotype_by_region_1 (index=6) is completed.
INFO: genotype_by_region_1 (index=7) is completed.
INFO: genotype_by_region_1 (index=8) is completed.
INFO: genotype_by_region_1 (index=9) is completed.
INFO: genotype_by_region_1 (index=10) is completed.
INFO: genotype_by_region_1 (index=14) is completed.
INFO: genotype_by_region_1 (index=13) is completed.
INFO: genotype_by_region_1 (index=11) is completed.
INFO: genotype_by_region_1 (index=15) is completed.
INFO: genotype_by_region_1 (index=12) is completed.
INFO: genotype_by_region_1 (index=16) is completed.
INFO: genotype_by_region_1 (index=17) is completed.
INFO: genotype_by_region_1 (index=18)

## Fine-mapping using individual level data

In [ ]:
sos run SuSiE.ipynb susie \
    --name protocol_example_protein \
    --genoFile output/protocol_example.protein.enhanced_cis_chr21_chr22_genotype_by_region/protocol_example.genotype.chr21_22.genotype_by_region_files.txt \
    --phenoFile output/phenotype/protocol_example.protein.region_list.txt \
                output/phenotype/protocol_example.protein.region_list.txt \
    --covFile output/covariate/protocol_example.protein.protocol_example.samples.protocol_example.genotype.chr21_22.pQTL.plink_qc.prune.pca.Marchenko_PC.gz \
              output/covariate/protocol_example.protein.protocol_example.samples.protocol_example.genotype.chr21_22.pQTL.plink_qc.prune.pca.Marchenko_PC.gz \
    --phenotype-names trait_A trait_B \
    --utils-R misc/xqtl_utils.R \
    --container containers/stephenslab.sif

**Please skip the analysis below because they need to be updated with the new workflow interface**

### SuSiE results post processing


In [ ]:
sos run pipeline/SuSiE_post_processing.ipynb susie_to_tsv \
    --cwd output/ADGWAS_finemapping_extracted/Bellenguez/ --rds_path `ls GWAS_Finemapping_Results/Bellenguez/ADGWAS2022*rds ` \
    --region-list ~/1300_hg38_EUR_LD_blocks_orig.tsv \
    --container containers/stephenslab.sif 

## Univariate SuSiF

In [ ]:
sos run pipeline/SuSiE.ipynb uni_fsusie \
    --genoFile /mnt/vast/hpc/csg/molecular_phenotype_calling/eqtl/output/dlpfc_tad_region_list_plink_files/ROSMAP_NIA_WGS.leftnorm.filtered.filtered.plink_files_list.txt  \
    --cwd output/f_susie_tad_haQTL_pos_check_pure_2/ --name haQTL \
    --region-list dlpfc_tad_list \
    --phenoFile phenotype_list \
    --covFile covar_list --prior mixture_normal_per_scale  --container containers/stephenslab.sif 

### fSuSiE results post processing


In [ ]:
sos run pipeline/SuSiE_post_processing.ipynb fsusie_to_tsv \
    --cwd output/f_susie_tad_haQTL_pos_check_pure_2 --rds_path `ls output/f_susie_tad_haQTL_pos_check_pure_2/cache/*rds ` \
    --region-list ../eqtl/dlpfc_tad_list \
    --container containers/stephenslab.sif 

sos run pipeline/SuSiE_post_processing.ipynb fsusie_to_tsv \
    --cwd output/f_susie_tad_haQTL_pos_check_pure_3 --rds_path `ls output/f_susie_tad_haQTL_pos_check_pure_3/cache/*rds ` \
    --region-list ../eqtl/dlpfc_tad_list \
    --container containers/stephenslab.sif


sos run pipeline/SuSiE_post_processing.ha fsusie_to_tsv \
    --cwd output/f_susie_tad_meQTL_pos_check_pure/ --rds_path `ls output/f_susie_tad_meQTL_pos_check_pure/cache/*rds ` \
    --region-list ../eqtl/dlpfc_tad_list \
    --container containers/stephenslab.sif

sos run pipeline/SuSiE_post_processing.ipynb fsusie_to_tsv \
    --cwd output/pQTL/ --rds_path `ls /mnt/vast/hpc/csg/molecular_phenotype_calling/pqtl_finemapping/output/susie_per_ID_tad/cache/*rds` \
    --region-list ../eqtl/dlpfc_tad_list \
    --container containers/stephenslab.sif


nohup sos run pipeline/SuSiE_post_processing.ipynb fsusie_to_tsv     --cwd output/f_susie_tad_haQTL_pos_limited_2/result --rds_path `ls output/f_susie_tad_haQT
L_pos_limited_2/cache/*rds`     --region-list ../eqtl/dlpfc_tad_list     --container containers/stephenslab.sif &

sos run pipeline/SuSiE_post_processing.ipynb fsusie_to_tsv \
    --cwd output/f_susie_tad_haQTL_pos_resume/ --rds_path `ls output/f_susie_tad_haQTL_pos_resume/cache/*rds` \
    --region-list ../eqtl/dlpfc_tad_list \
    --container containers/stephenslab.sif

sos run pipeline/SuSiE_post_processing.ipynb fsusie_to_tsv \
    --cwd output/f_susie_tad_meQTL_pos_resume/ --rds_path `ls output/f_susie_tad_meQTL_pos_resume/cache/*rds` \
    --region-list ../eqtl/dlpfc_tad_list \
    --container containers/stephenslab.sif


sos run pipeline/SuSiE_post_processing.ipynb fsusie_to_tsv \
    --cwd output/f_susie_tad_haQTL_pos_resume/result --rds_path `ls output/f_susie_tad_haQTL_pos_resume/cache/*rds` \
    --region-list ../eqtl/dlpfc_tad_list \
    --container containers/stephenslab.sif &


sos run pipeline/SuSiE_post_processing.ipynb fsusie_to_tsv \
    --cwd output/test --rds_path `ls output/f_susie_tad_haQTL_pos_check_pure_2/cache/*rds | head -1 ` \
    --region-list ../eqtl/dlpfc_tad_list \
    --container /home/hs3163/test_sif/stephenslab.sif


## haQTL

In [ ]:
sos run pipeline/SuSiE_post_processing.ipynb fsusie_to_tsv \
    --cwd output/f_susie_tad_haQTL_pos_check_pure_2/result --rds_path `ls output/f_susie_tad_haQTL_pos_check_pure_2/cache/*rds ` \
    --region-list ../eqtl/dlpfc_tad_list \
    --container containers/stephenslab.sif  &


sos run pipeline/SuSiE_post_processing.ipynb fsusie_to_tsv \
    --cwd output/f_susie_tad_haQTL_resume/result --rds_path `ls output/f_susie_tad_haQTL_resume/cache/*rds` \
    --region-list ../eqtl/dlpfc_tad_list \
    --container containers/stephenslab.sif &


nohup sos run pipeline/SuSiE_post_processing.ipynb fsusie_to_tsv   \
    --cwd output/f_susie_tad_haQTL_pos_limited_2/result \
    --rds_path `ls output/f_susie_tad_haQTL_pos_limited_2/cache/*rds`   \
    --region-list ../eqtl/dlpfc_tad_list     --container containers/stephenslab.sif &




## mQTL

In [ ]:
nohup sos run pipeline/SuSiE_post_processing.ipynb fsusie_to_tsv \
    --cwd output/f_susie_tad_meQTL_pos_2/result --rds_path `ls output/f_susie_tad_meQTL_pos_2/cache/*rds ` \
    --region-list ../eqtl/dlpfc_tad_list \
    --container containers/stephenslab.sif  &

nohup sos run pipeline/SuSiE_post_processing.ipynb fsusie_to_tsv \
    --cwd output/f_susie_tad_meQTL_pos_resume/result --rds_path `ls output/f_susie_tad_meQTL_pos_resume/cache/*rds ` \
    --region-list ../eqtl/dlpfc_tad_list \
    --container containers/stephenslab.sif  &

nohup sos run pipeline/SuSiE_post_processing.ipynb fsusie_to_tsv \
    --cwd output/f_susie_tad_meQTL_pos_limited_2/result --rds_path `ls output/f_susie_tad_meQTL_pos_limited_2/cache/*rds ` \
    --region-list ../eqtl/dlpfc_tad_list \
    --container containers/stephenslab.sif  &



## Plotting susie

### PIP land scape plot

In [ ]:
sos run pipeline/SuSiE_post_processing.ipynb susie_pip_landscape_plot \
    --cwd output/test/ --plot_list plot_recipe_668 --annot_tibble ~/Annotatr_builtin_annotation_tibble.tsv -s force --container containers/stephenslab.sif  &

sos run pipeline/SuSiE_post_processing.ipynb susie_pip_landscape_plot \
    --cwd output/test/ --plot_list plot_recipe_ADGWAS_uni --annot_tibble ~/Annotatr_builtin_annotation_tibble.tsv -s force --container containers/stephenslab.sif  &

sos run pipeline/SuSiE_post_processing.ipynb susie_pip_landscape_plot \
    --cwd output/1182/ --plot_list plot_recipe_1182 --annot_tibble ~/Annotatr_builtin_annotation_tibble.tsv -s force --container containers/stephenslab.sif  &

In [ ]:
sos run pipeline/SuSiE_post_processing.ipynb susie_pip_landscape_plot \
    --cwd output/test_3/ --plot_list plot_recipe --annot_tibble ~/Annotatr_builtin_annotation_tibble.tsv -s force --container containers/stephenslab.sif  &

In [ ]:
sos run pipeline/SuSiE_post_processing.ipynb susie_pip_landscape_plot \
    --cwd output/5g/ --plot_list recipe_5gene --annot_tibble ~/Annotatr_builtin_annotation_tibble.tsv -s force --container containers/stephenslab.sif  &

### UpSetR plot 

In [ ]:
sos run pipeline/SuSiE_post_processing.ipynb susie_upsetR_plot \
    --cwd output/updated_mQTL/ --plot_list UpsetR_recipe -s force --container containers/stephenslab.sif &
sos run pipeline/SuSiE_post_processing.ipynb susie_upsetR_plot \
    --cwd output/updated_16/ --plot_list UpsetR_recipe_16 -s force --container containers/stephenslab.sif &

In [ ]:
sos run pipeline/SuSiE_post_processing.ipynb susie_upsetR_cs_plot \
    --cwd output/updated_mQTL/ --plot_list UpsetR_recipe_1 -s force --trait_to_select 3 --container containers/stephenslab.sif &
sos run pipeline/SuSiE_post_processing.ipynb susie_upsetR_cs_plot \
    --cwd output/updated_16/ --plot_list UpsetR_recipe_16  -s force --trait_to_select 3 --container containers/stephenslab.sif &

sos run pipeline/SuSiE_post_processing.ipynb susie_upsetR_cs_plot \
    --cwd output/rerun/ --plot_list UpsetR_recipe_16_rerun  -s force --trait_to_select 3 --container containers/stephenslab.sif &

sos run pipeline/SuSiE_post_processing.ipynb susie_upsetR_cs_plot \
    --cwd output/rerun/ --plot_list UpsetR_recipe_16_rerun  -s force --trait_to_select 2 --container containers/stephenslab.sif &

sos run pipeline/SuSiE_post_processing.ipynb susie_upsetR_cs_plot \
    --cwd output/rerun/ --plot_list UpsetR_recipe_16_rerun  -s force --trait_to_select 1 --container containers/stephenslab.sif &

## FSuSiE estimated effect plot

In [ ]:
sos run pipeline/SuSiE_post_processing.ipynb fsusie_extract_effect \
    --cwd output/all_haQTL_effect/ --rds_path /mnt/vast/hpc/csg/molecular_phenotype_calling/QTL_fine_mapping/output/definitive_haQTL//*.rds \
    --container containers/stephenslab.sif

sos run pipeline/SuSiE_post_processing.ipynb fsusie_extract_effect \
    --cwd output/all_meQTL_effect/ --rds_path /mnt/vast/hpc/csg/molecular_phenotype_calling/QTL_fine_mapping/output/definitive_mQTL/*rds \
    --container containers/stephenslab.sif -s build 

In [ ]:
nohup sos run pipeline/SuSiE_post_processing.ipynb fsusie_extract_effect \
    --cwd output/rerun/all_haQTL_effect/ --rds_path `ls output/rerun/cache/*haqtl*rds | grep -v -f empty_rds ` \
    --container containers/stephenslab.sif -s build  &

nohup sos run pipeline/SuSiE_post_processing.ipynb fsusie_extract_effect \
    --cwd output/rerun/all_meQTL_effect/ --rds_path `ls output/rerun/cache/*mqtl*rds | grep -v -f empty_rds ` \
    --container containers/stephenslab.sif -s build &

## FSuSiE credible band

In [ ]:
sos run pipeline/SuSiE_post_processing.ipynb fsusie_affected_region \
    --cwd output/definitive_haQTL/affected_region --rds_path /mnt/vast/hpc/csg/molecular_phenotype_calling/QTL_fine_mapping/output/definitive_haQTL/*.rds \
    --container containers/stephenslab.sif

In [ ]:
sos run pipeline/SuSiE_post_processing.ipynb fsusie_affected_region \
    --cwd output/definitive_mQTL/affected_region --rds_path /mnt/vast/hpc/csg/molecular_phenotype_calling/QTL_fine_mapping/output/definitive_mQTL/*.rds \
    --container containers/stephenslab.sif

In [ ]:
nohup sos run pipeline/SuSiE_post_processing.ipynb fsusie_affected_region    \
    --cwd output/definitive_mQTL/affected_region \
    --rds_path /mnt/vast/hpc/csg/molecular_phenotype_calling/QTL_fine_mapping/output/definitive_mQTL/*.rds    \
    --container containers/stephenslab.sif -J 300 -q csg2 -c csg2.yml  &

In [ ]:
nohup sos run pipeline/SuSiE_post_processing.ipynb fsusie_affected_region \
    --cwd output/rerun/affected_region/mQTL --rds_path `ls output/rerun/cache/*mqtl*rds | grep -v -f empty_rds ` \
    --container containers/stephenslab.sif &

nohup sos run pipeline/SuSiE_post_processing.ipynb fsusie_affected_region \
    --cwd output/rerun/affected_region/haQTL --rds_path `ls output/rerun/cache/*haqtl*rds | grep -v -f empty_rds ` \
    --container containers/stephenslab.sif &